<a href="https://colab.research.google.com/github/olaniyi2oguns/Absenteeism_data_ML_Algorithm/blob/main/Absenteeism_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoostM

### Import necessary Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score, precision_score,recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

### Load the data

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/DATA SCIENCE/Absenteeism_processed_data.csv')

print(dataset.head())

   Reason_1  Reason_2  Reason_3  Reason_4  Month Value  Day of the Week  \
0         0         0         0         1            7                1   
1         0         0         0         0            7                1   
2         0         0         0         1            7                2   
3         0         0         0         0            7                3   
4         0         0         0         1            7                3   

         Date  Transportation Expense  Distance to Work  Age  \
0  2015-07-07                     289                36   33   
1  2015-07-14                     118                13   50   
2  2015-07-15                     179                51   38   
3  2015-07-16                     279                 5   39   
4  2015-07-23                     289                36   33   

   Daily Work Load Average  Body Mass Index  Education  Children  Pets  \
0                  239.554               30          0         2     1   
1               

### Preprocessed the target(Absenteeism Time in Hours)

In [4]:
dataset['Absenteeism Time in Hours'].median() #Find the median of the values in this column in order to group them in to 1s and 0s

3.0

In [5]:
target = np.where(dataset['Absenteeism Time in Hours'] > 3, 1, 0)

print(target)

[1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1
 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 0 0 1 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 1 0 0 0 1 0
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0
 0 1 0 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 0 1
 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1
 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 1
 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 0 0
 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0
 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1
 0 1 1 1 1 1 1 1 1 1 0 1 

### Add target to the dataframe

In [6]:
dataset['Absenteeism'] = target

In [7]:
dataset.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Absenteeism
0,0,0,0,1,7,1,2015-07-07,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,2015-07-14,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,2015-07-15,179,51,38,239.554,31,0,0,0,2,0
3,0,0,0,0,7,3,2015-07-16,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,2015-07-23,289,36,33,239.554,30,0,2,1,2,0


In [8]:
new_dataset = dataset.drop(['Date', 'Absenteeism Time in Hours'], axis = 1)

In [9]:
new_dataset

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,0,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,5,2,179,22,40,237.656,22,1,2,0,1
696,0,0,0,0,5,2,225,26,28,237.656,24,0,1,2,0
697,0,0,0,0,5,3,330,16,28,237.656,25,1,0,0,1
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0,0


###  Define the feature and target variable

In [10]:
X = new_dataset.iloc[:, :-1].values   # the feature variables
y = new_dataset.iloc[:, -1].values    # the target variable

In [11]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [4,5,6,7,8,9,10,12,13])], remainder='passthrough')
X = ct.fit_transform(X)

### Splitting the Dataset to train and test sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [13]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

### Train the Logistic regression model

In [14]:
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [15]:
classifier.score(X_train, y_train)

0.9482142857142857

### Prediction

In [16]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [1 0]
 [1 0]
 [0 1]
 [0 0]
 [0 1]
 [1 1]
 [1 1]
 [0 1]
 [1 0]
 [1 1]
 [0 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [1 1]
 [0 1]
 [1 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [0 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 1]
 [1 0]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]]


### Confusion matrix and accuracy

In [17]:
accuracy_score(y_test, y_pred)

0.6642857142857143

In [18]:
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

In [19]:
print("Accuracy", accuracy_score)
print("Confusion Matrix", cm)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

Accuracy <function accuracy_score at 0x7e03cf7793f0>
Confusion Matrix [[50 22]
 [25 43]]
F1 Score: 0.6634443250984603
Precision: 0.6641025641025641
Recall: 0.6633986928104575


### K-Fold Cross validation

In [20]:
accuracies = cross_val_score(estimator =classifier, X = X_train, y = y_train, cv=10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 71.96 %
Standard Deviation: 4.52 %
